# Can run without errors in google colab
### hardware specs are too low for deep learning


In [1]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline

In [2]:
url = 'https://raw.githubusercontent.com/codebasics/deep-learning-keras-tf-tutorial/master/6_gradient_descent/insurance_data.csv'

df = pd.read_csv(url)
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [3]:
x = df[['age', 'affordibility']]
x.head()

,age,affordibility
0,22,1
1,25,0
2,47,1
3,52,0
4,46,1


In [4]:
y = df.bought_insurance

In [5]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=42)

In [6]:
len(df),len(x_train),len(x_test)

(28, 22, 6)

In [7]:
x_train_scaled = x_train.copy()
x_train_scaled['age'] = x_train_scaled['age']/100
x_test_scaled = x_test.copy()
x_test_scaled['age'] = x_test_scaled['age']/100

In [8]:
x_train_scaled.head()

,age,affordibility
17,0.58,1
22,0.40,1
11,0.28,1
13,0.29,0
15,0.55,1


In [9]:
model = keras.Sequential([
    keras.layers.Dense(1, input_shape=(2,),activation='sigmoid',kernel_initializer='ones', bias_initializer='zeros')  # neuron 1, input 2 neurons
])

model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',       # the same as logloss function
    metrics = ['accuracy']
)

model.fit(x_train_scaled, y_train, epochs=5000)

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 9ms/step - loss: 0.5605 - accuracy: 0.8636
Epoch 2502/5000
1/1 [==============================] - 0s 10ms/step - loss: 0.5605 - accuracy: 0.8636
Epoch 2503/5000
1/1 [==============================] - 0s 8ms/step - loss: 0.5605 - accuracy: 0.8636
Epoch 2504/5000
1/1 [==============================] - 0s 5ms/step - loss: 0.5604 - accuracy: 0.8636
Epoch 2505/5000
1/1 [==============================] - 0s 9ms/step - loss: 0.5604 - accuracy: 0.8636
Epoch 2506/5000
1/1 [==============================] - 0s 5ms/step - loss: 0.5603 - accuracy: 0.8636
Epoch 2507/5000
1/1 [==============================] - 0s 4ms/step - loss: 0.5603 - accuracy: 0.8636
Epoch 2508/5000
1/1 [==============================] - 0s 7ms/step - loss: 0.5603 - accuracy: 0.8636
Epoch 2509/5000
1/1 [==============================] - 0s 9ms/step - loss: 0.5602 - accuracy: 0.8636
Epoch 2510/5000
1/1 [==============================] - 

In [10]:
model.evaluate(x_test_scaled, y_test)

1/1 [==============================] - 0s 186ms/step - loss: 0.2647 - accuracy: 1.0000


[0.2647029757499695, 1.0]

In [11]:
y_predicted = model.predict(x_test_scaled)

In [12]:
y_predicted[:5]

array([[0.81626725],
       [0.7576756 ],
       [0.8236744 ],
       [0.18819532],
       [0.3855912 ]], dtype=float32)

In [13]:
x_test_scaled[:5]

,age,affordibility
9,0.61,1
25,0.54,1
8,0.62,1
21,0.26,0
0,0.22,1


In [14]:
y_test[:5]

9     1
25    1
8     1
21    0
0     0
Name: bought_insurance, dtype: int64

In [15]:
coef, intercept = model.get_weights()

In [16]:
coef, intercept

(array([[5.018314 ],
        [1.1966292]], dtype=float32), array([-2.766541], dtype=float32))

In [17]:
import math

def sigmoid(x):
  return 1 / (1 + np.exp(-x))

sigmoid(18)

0.9999999847700205

In [18]:
def prediction_func(age, affordibility):
  weighted_sum = coef[0]*age + coef[1]* affordibility + intercept
  return sigmoid(weighted_sum)

In [19]:
prediction_func(.18, 0)

array([0.13432531], dtype=float32)

In [20]:
prediction_func(.47, 1)

array([0.68755126], dtype=float32)

In [29]:
def log_loss(y_true, y_predicted):
  epsilon = 1e-15
  y_predicted_new = [max(i, epsilon) for i in y_predicted]
  y_predicted_new = [min(i, 1-epsilon) for i in y_predicted_new]
  y_predicted_new = np.array(y_predicted_new)
  return -np.mean(y_true*np.log(y_predicted_new) + (1-y_true) * np.log(1 - y_predicted_new))

In [43]:
def sigmoid_numpy(x):
  return 1 / (1 + np.exp(-x))

sigmoid_numpy(np.array([12,0,1]))

array([0.99999386, 0.5       , 0.73105858])

In [37]:
def gradient_descent(age, affordability, y_true, epochs, loss_threshold):
  w1=w2=1
  bias = 0
  rate = 0.5
  n = len(age)

  for i in range(epochs):
    weighted_sum = w1 * age + w2 * affordability + bias
    y_predicted = sigmoid_numpy(weighted_sum)

    loss = log_loss(y_true, y_predicted)

    dw1 = 1/n* np.dot(np.transpose(age),(y_predicted-y_true))
    dw2 = 1/n* np.dot(np.transpose(affordability),(y_predicted-y_true))

    bias_d = np.mean(y_predicted-y_true)

    w1 = w1 - rate * dw1
    w2 = w2 - rate * dw2
    bias = bias - rate * bias_d

    print(f'epoch {i}, w1:{w1}, w2:{w2}, bias:{bias}, loss: {loss}')

  return w1, w2, bias

In [40]:
gradient_descent(x_train_scaled['age'], x_train_scaled['affordibility'], y_train, 5000, 0.4923)

epoch 0, w1:0.9736899318847281, w2:0.931388810977659, bias:-0.11748951666770448, loss: 0.7428288579142563
epoch 1, w1:0.9536535852311093, w2:0.8740290167758512, bias:-0.21881533456146035, loss: 0.7072146449948488
epoch 2, w1:0.9393731039296968, w2:0.8271852202997496, bias:-0.3053620401943441, loss: 0.6814881914786812
epoch 3, w1:0.930193258899806, w2:0.7897792032048467, bias:-0.37884372361582785, loss: 0.6633428084673968
epoch 4, w1:0.9254091137248937, w2:0.7605726653866934, bias:-0.441082368200183, loss: 0.650742850709519
epoch 5, w1:0.9243325693598606, w2:0.738313053647322, bias:-0.4938425798625155, loss: 0.6420508089402462
epoch 6, w1:0.9263332963572349, w2:0.7218280753843739, bias:-0.5387319906498417, loss: 0.6360356979531206
epoch 7, w1:0.9308580975636879, w2:0.7100747303660235, bias:-0.5771558825717441, loss: 0.631816485354411
epoch 8, w1:0.9374354910317361, w2:0.7021560855322683, bias:-0.6103083840841516, loss: 0.6287844495353145
epoch 9, w1:0.9456716791005844, w2:0.697318549631

(11.39068072634572, 1.8530842457781869, -5.8938873282238164)